In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
image_train_gen = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255.,
                                                         horizontal_flip=True,
                                                         fill_mode="nearest",
                                                         validation_split=0.2
                                                         )

image_test_gen = keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.2,
                                                         height_shift_range=0.2,
                                                         rescale=1 / 255.,
                                                         horizontal_flip=True,
                                                         fill_mode="nearest"
                                                         )

In [ ]:
train_images = image_train_gen.flow_from_directory(
    directory="scanner/train/",
    color_mode="rgb",
    class_mode="binary",
    subset='training',
    seed=42
    )

Found 42043 images belonging to 2 classes.


In [ ]:
valid_images = image_train_gen.flow_from_directory(
    directory="scanner/train/",
    color_mode="rgb",
    class_mode="binary",
    subset='validation',
    seed=42
    )

Found 10510 images belonging to 2 classes.


In [ ]:
test_images = image_test_gen.flow_from_directory(
    directory="scanner/test/",
    color_mode="rgb",
    class_mode="binary",
    seed=42
    )

Found 13452 images belonging to 2 classes.


In [ ]:
train_images.image_shape

(256, 256, 3)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[256,256,3]))
model.add(keras.layers.Dense(1000, activation="relu"))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))  # "sigmoid" for binary classification

In [ ]:
model.compile(loss="binary_crossentropy",   ## for binary classification
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("models/model-{epoch:02d}-{val_accuracy:.2f}.hdf5",  monitor='val_accuracy', save_best_only=True)

In [ ]:
history = model.fit(train_images, epochs=100, validation_data=valid_images, callbacks=[early_stopping_cb, checkpoint_cb])

Epoch 1/100
1314/1314 [==============================] - 677s 513ms/step - loss: 0.2486 - accuracy: 0.9168 - val_loss: 0.1932 - val_accuracy: 0.9372
Epoch 2/100
1314/1314 [==============================] - 670s 510ms/step - loss: 0.1418 - accuracy: 0.9547 - val_loss: 0.1402 - val_accuracy: 0.9586
Epoch 3/100
1314/1314 [==============================] - 670s 510ms/step - loss: 0.1112 - accuracy: 0.9659 - val_loss: 0.1192 - val_accuracy: 0.9621
Epoch 4/100
1314/1314 [==============================] - 669s 509ms/step - loss: 0.1125 - accuracy: 0.9648 - val_loss: 0.0985 - val_accuracy: 0.9668
Epoch 5/100
1314/1314 [==============================] - 670s 510ms/step - loss: 0.0933 - accuracy: 0.9714 - val_loss: 0.0976 - val_accuracy: 0.9710
Epoch 6/100
1314/1314 [==============================] - 666s 507ms/step - loss: 0.0870 - accuracy: 0.9725 - val_loss: 0.1424 - val_accuracy: 0.9590
Epoch 7/100
1314/1314 [==============================] - 669s 509ms/step - loss: 0.0839 - accuracy: 0.9731